In [41]:
pip install lxml

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
import pandas as pd
from scipy import signal
import numpy as np
import matplotlib as plt
import time

import matplotlib.pyplot as plt
import warnings
import re

# Ignore all warnings
warnings.filterwarnings("ignore")

In [43]:
Combine = pd.read_csv("data/combine.csv")
Team_Stats = pd.read_csv("data/team_stats_2003_2023.csv")

In [44]:
Team_Stats = Team_Stats[Team_Stats["year"] != 2023]

## Webscrape all of the years for from pro-football reference that we are using

In [45]:
website_begining = 'https://www.pro-football-reference.com/years/'
years = Team_Stats["year"].unique()
years = years
positions = ['passing', 'rushing', 'receiving']
years

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [8]:
website_begining = 'https://www.pro-football-reference.com/years/'
years = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
positions = ['passing', 'rushing', 'receiving']
for pos in positions:
    print(pos)
    for year in years:
        link = website_begining + str(year) +'/'+ pos + '.htm'
        
        df = pd.read_html(link)
        
        csv_name = 'data/'+str(year) + '_' + pos + '.csv'
        df[0].to_csv(csv_name, sep=',', index=False, encoding='utf-8')
        time.sleep(4.2) #wait four seconds
        print(year)

passing
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
rushing
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
receiving
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022


## Concat the tables together to put them all into one data frame and take out unnessary data from player name

In [46]:
passing = pd.read_csv('data/2003_passing.csv')

passing["Year"] = years[0]
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_passing.csv')
    temp["Year"] = year
    passing = pd.concat([passing, temp])
    
passing = passing[passing["Pos"] == "QB"]
passing.reset_index(drop=True, inplace=True)

In [47]:
index = 0
for name in passing["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    passing["Player"][index] = front + back
    
    index += 1

In [48]:
rushing = pd.read_csv('data/2003_rushing.csv')

rushing["Year"] = years[0]

for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_rushing.csv')
    temp["Year"] = year
    rushing = pd.concat([rushing, temp])
    
    
rushing.columns = rushing.iloc[0]
rushing = rushing.drop(0)
rushing.rename(columns={2003: 'Year'}, inplace=True)

rushing = rushing[rushing["Pos"] == "RB"]
rushing.reset_index(drop=True, inplace=True)

In [49]:
index = 0
for name in rushing["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    rushing["Player"][index] = front + back
    
    index += 1


In [50]:
receiving = pd.read_csv('data/2003_receiving.csv')

receiving["Year"] = years[0]
for year in years[1:]:
    temp = pd.read_csv('data/' + str(year) + '_receiving.csv')
    temp["Year"] = year
    receiving = pd.concat([receiving, temp])
    
receiving = receiving[receiving["Pos"] == "WR"]
receiving.reset_index(drop=True, inplace=True)

In [51]:
index = 0
for name in receiving["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    receiving["Player"][index] = front + back
    
    index += 1
    

In [52]:
print("Begining Shapes of DataFrames:\n")
print('Passing Observations:', passing.shape[0], '       Passing Parameters:', passing.shape[1] - 2, '\n')
print('Rushing Observations:', rushing.shape[0], '       Rushing Parameters:', rushing.shape[1] - 2, '\n')
print('Receiving Observations:', receiving.shape[0], '     Receiving Parameters:', receiving.shape[1] - 2, '\n')

Begining Shapes of DataFrames:

Passing Observations: 1475        Passing Parameters: 31 

Rushing Observations: 2671        Rushing Parameters: 15 

Receiving Observations: 4020      Receiving Parameters: 19 



## Putting Combine Data into Positional Dfs

In [53]:
for col in Combine.columns:
    passing[col] = 0
    rushing[col] = 0
    receiving[col] = 0

In [54]:
for col in Combine.columns:
    passing[col] = 0
passing_names = passing["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in passing_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        passing.loc[passing["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(passing_names))) + "% of the players had Combine data")
passing = passing[passing["hsCountry"] != 0]

70% of the players had Combine data


In [55]:
rushing_names = rushing["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in rushing_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        rushing.loc[rushing["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(rushing_names))) + "% of the players had Combine data")
rushing = rushing[rushing["hsCountry"] != 0]

57% of the players had Combine data


In [56]:
receiving_names = receiving["Player"].unique()
combine_names = Combine["nameFull"].unique()

has_data = 0
for name in receiving_names:
    if name in combine_names:
        combine_values = Combine[Combine["nameFull"]==name].values.tolist()[0]
        receiving.loc[receiving["Player"] == name, Combine.columns] = combine_values
        
        has_data += 1
    
    
print(str(int(100*has_data/len(receiving_names))) + "% of the players had Combine data")
receiving = receiving[receiving["hsCountry"] != 0]

49% of the players had Combine data


In [57]:
passing.drop(columns=['Rk'], inplace=True)
rushing.drop(columns=['Rk'], inplace=True)
receiving.drop(columns=['Rk'], inplace=True)

## Putting the team data into Positional Dfs

In [58]:
index = 0
for team in Team_Stats["team"]:
    team_split = team.split(" ")
    simple = team_split[-1]
    
    Team_Stats["team"][index] = simple
    index += 1

In [59]:
Team_Stats["team"] = Team_Stats["team"].replace("Redskins", "Commanders")
Team_Stats["team"] = Team_Stats["team"].replace("Team", "Commanders")

In [60]:
df_list = [passing, rushing, receiving]

In [61]:
for df in df_list:
    df["Tm"] = df["Tm"].replace("MIN", "Vikings")
    df["Tm"] = df["Tm"].replace("KAN", "Chiefs")
    df["Tm"] = df["Tm"].replace("IND", "Colts")
    df["Tm"] = df["Tm"].replace("DEN", "Broncos")
    df["Tm"] = df["Tm"].replace("GNB", "Packers")
    df["Tm"] = df["Tm"].replace("STL", "Rams")#
    df["Tm"] = df["Tm"].replace("PHI", "Eagles")
    df["Tm"] = df["Tm"].replace("NOR", "Steelers")#
    df["Tm"] = df["Tm"].replace("NWE", "Patriots")
    df["Tm"] = df["Tm"].replace("DAL", "Cowboys")

    df["Tm"] = df["Tm"].replace("HOU", "Texans")
    df["Tm"] = df["Tm"].replace("OAK", "Raiders")
    df["Tm"] = df["Tm"].replace("SDG", "Chargers")
    df["Tm"] = df["Tm"].replace("DET", "Lions")
    df["Tm"] = df["Tm"].replace("JAX", "Jaguars")
    df["Tm"] = df["Tm"].replace("BUF", "Bills")
    df["Tm"] = df["Tm"].replace("CIN", "Bengals")
    df["Tm"] = df["Tm"].replace("NYJ", "Jets")
    df["Tm"] = df["Tm"].replace("PIT", "Steelers")
    df["Tm"] = df["Tm"].replace("BAL", "Ravens")

    df["Tm"] = df["Tm"].replace("ARI", "Cardinals")
    df["Tm"] = df["Tm"].replace("ATL", "Falcons")
    df["Tm"] = df["Tm"].replace("IND", "Colts")
    df["Tm"] = df["Tm"].replace("SFO", "49ers")
    df["Tm"] = df["Tm"].replace("CLE", "Browns")
    df["Tm"] = df["Tm"].replace("TEN", "Titans")
    df["Tm"] = df["Tm"].replace("WAS", "Commanders")
    df["Tm"] = df["Tm"].replace("NYG", "Giants")
    df["Tm"] = df["Tm"].replace("CHI", "Bears")
    df["Tm"] = df["Tm"].replace("TAM", "Buccaneers")

    df["Tm"] = df["Tm"].replace("MIA", "Dolphins")
    df["Tm"] = df["Tm"].replace("CAR", "Panthers")
    df["Tm"] = df["Tm"].replace("SEA", "Seahawks")
    df["Tm"] = df["Tm"].replace("2TM", "Take_out")
    df["Tm"] = df["Tm"].replace("LAR", "Rams")
    df["Tm"] = df["Tm"].replace("LAC", "Chargers")
    df["Tm"] = df["Tm"].replace("LVR", "Raiders")
    
    df["Tm"] = df["Tm"].replace("Team", "Commanders")

In [62]:
passing = passing[(passing["Tm"] != "Take_out") & (passing["Tm"] != "3TM")]
rushing = rushing[(rushing["Tm"] != "Take_out") & (rushing["Tm"] != "3TM" )]
receiving = receiving[(receiving["Tm"] != "Take_out") & (receiving["Tm"] != "3TM")]

In [63]:
pos_team_cols =Team_Stats.columns[2:]
for col in pos_team_cols:
    passing["team_" + col] = 0
    rushing["team_" + col] = 0
    receiving["team_" + col] = 0
    

passing.reset_index(drop=True, inplace=True)
rushing.reset_index(drop=True, inplace=True)
receiving.reset_index(drop=True, inplace=True)

In [64]:
df_list = [passing, rushing, receiving]
team_years = Team_Stats["year"].unique()
for df in df_list:
    for year in team_years:
        for team in Team_Stats["team"].unique():
            team_values = Team_Stats[(Team_Stats["team"]==team) & (Team_Stats["year"]==year)].values.tolist()[0]
            df.loc[(df["Tm"] == team) & (df["Year"] == year), "team_" + Team_Stats.columns[2:]] = team_values[2:]

## Taking out unnessary Columns

In [65]:
df_list = [passing, rushing, receiving]
questionable_cols = []
for df in df_list:
    question = 0
    question_temp = []
    for col in df.columns:
        if type(df[col][0]) != float or type(df[col][0]) != int:
            if col != 'Player' and col != 'Tm':
                try:
                    df[col] = df[col].astype(float)
                except:
                    question_temp.append(col)
                    question += 1

    print(100*question/len(df.columns), "% was questionable")
    print(question_temp)
    questionable_cols.append(question_temp)

18.0 % was questionable
['Pos', 'QBrec', 'combinePosition', 'nameFirst', 'nameLast', 'nameFull', 'position', 'nflId', 'college', 'dob', 'playerProfileUrl', 'homeCity', 'homeState', 'homeCountry', 'highSchool', 'hsCity', 'hsState', 'hsCountry']
20.238095238095237 % was questionable
['Pos', 'combinePosition', 'nameFirst', 'nameLast', 'nameFull', 'position', 'nflId', 'college', 'dob', 'playerProfileUrl', 'homeCity', 'homeState', 'homeCountry', 'highSchool', 'hsCity', 'hsState', 'hsCountry']
20.454545454545453 % was questionable
['Pos', 'Ctch%', 'combinePosition', 'nameFirst', 'nameLast', 'nameFull', 'position', 'nflId', 'college', 'dob', 'playerProfileUrl', 'homeCity', 'homeState', 'homeCountry', 'highSchool', 'hsCity', 'hsState', 'hsCountry']


In [66]:
passing = passing.drop(columns = [questionable_cols[0]][0])
rushing = rushing.drop(columns = [questionable_cols[1]][0])
receiving = receiving.drop(columns = [questionable_cols[2]][0])

## Deal with Nan values

In [67]:
df_list = [passing, rushing, receiving]
nan_columns = []
for df in df_list:
    temp_nan = []
    total = 0
    for col in df.columns:
        column_nan_percentage = df[col].isna().mean()
        if column_nan_percentage >= 0.05:
            print(col, column_nan_percentage*100)
            total += 1
            temp_nan.append(col)
    nan_columns.append(temp_nan)
    print(total/len(df.columns))

4QC 45.950554134697356
GWD 45.950554134697356
QBR 15.515771526001704
combineHand 88.57630008525149
combineArm 88.7468030690537
combine40yd 6.138107416879795
combineVert 23.017902813299234
combineBench 94.8849104859335
combineShuttle 27.621483375959077
combineBroad 24.21142369991475
combine3cone 31.54305200341006
combine60ydShuttle 98.80647911338448
combineWonderlic 30.434782608695656
team_mov 48.4228473998295
team_ties 51.57715260017051
0.18292682926829268
combineHand 92.90780141843972
combineArm 93.14420803782507
combine40yd 6.796690307328605
combineVert 11.93853427895981
combineBench 18.55791962174941
combineShuttle 33.92434988179669
combineBroad 15.425531914893616
combine3cone 39.00709219858156
combine60ydShuttle 97.04491725768321
combineWonderlic 93.91252955082743
team_mov 50.945626477541374
team_ties 49.054373522458626
0.1791044776119403
combineHand 88.15331010452961
combineArm 88.93728222996515
combine40yd 6.837979094076655
combineVert 13.937282229965156
combineBench 48.867595818

In [68]:
passing = passing.drop(columns = [nan_columns[0]][0])
rushing = rushing.drop(columns = [nan_columns[1]][0])
receiving = receiving.drop(columns = [nan_columns[2]][0])

## Replace rest of Nan with column average

In [69]:
df_list = [passing, rushing, receiving]
for df in df_list:
    total = 0
    for col in df.columns:
        column_nan_percentage = df[col].isna().mean()
        if column_nan_percentage > 0:
            
            column_average = df[col].mean()


            df[col] = df[col].fillna(column_average)
            print(col)

Y/C
ageAtDraft
ageAtDraft
Tgt
Y/R
Succ%
Y/Tgt
Fmb
ageAtDraft


# WebScrape Fantasy Football Rankings from Pro Football Reference

In [70]:
website_begining = 'https://www.pro-football-reference.com/years/'
years = Team_Stats["year"].unique()
years_fantasy = years + 1
positions = ['passing', 'rushing', 'receiving']
years_fantasy

array([2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [196]:
for year in years_fantasy:
    link = website_begining + str(year) + '/fantasy.htm'

    df = pd.read_html(link)

    csv_name = 'data/'+str(year) + '_Fantasy_Score.csv'
    df[0].to_csv(csv_name, sep=',', index=False, encoding='utf-8')
    time.sleep(4.2) #wait four seconds
    print(year)

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [71]:
Fantasy = pd.read_csv('data/2003_Fantasy_Score.csv')

Fantasy["Year"] = years_fantasy[0]
for year in years_fantasy[1:]:
    temp = pd.read_csv('data/' + str(year) + '_Fantasy_Score.csv')
    temp["Year"] = year
    Fantasy = pd.concat([Fantasy, temp])
    
    
    
Fantasy.columns = Fantasy.iloc[0]
Fantasy = Fantasy.drop(0)
Fantasy.rename(columns={years_fantasy[0]: 'Year'}, inplace=True)


Fantasy = Fantasy[(Fantasy["FantPos"] == "QB") | (Fantasy["FantPos"] == "WR") | (Fantasy["FantPos"] == "RB")]

Fantasy.reset_index(drop=True, inplace=True)

In [72]:
index = 0
for name in Fantasy["Player"]:
    front = name[:-2]
    
    back = re.sub(r'[^a-zA-Z]', '', name[-2:])
    
    Fantasy["Player"][index] = front + back
    
    index += 1
    
keep_columns = ['Player', 'PosRank', 'Year']

Fantasy = Fantasy[keep_columns]

# Add Rankings to Dfs

In [73]:

passing["Next_Year_PosRank"] = 0
rushing["Next_Year_PosRank"] = 0
receiving["Next_Year_PosRank"] = 0
    

passing.reset_index(drop=True, inplace=True)
rushing.reset_index(drop=True, inplace=True)
receiving.reset_index(drop=True, inplace=True)

In [74]:

fantasy_names = Fantasy["Player"].unique()
total = 0
has_data = 0
for df in df_list:
    df_names = df["Player"].unique()
    total += len(df_names)
    for name in df_names:
        if name in fantasy_names:
            for year in years:
                try:
                    fantasy_value = int(Fantasy[(Fantasy["Player"] == name) & (Fantasy["Year"] == year + 1)]["PosRank"])
                
                    df.loc[(df["Player"] == name) & (df["Year"] == year), ['Next_Year_PosRank']] = fantasy_value
        
                #df[(df["Player"] == name) & (df["Year"] == year)]['PosRank'] = fantasy_value

                    has_data += 1
                    total += 1
                except:
                    pass
    
    
print(str(int(100*has_data/total)) + "% of the players had PosRank data")

82% of the players had PosRank data


In [75]:
passing = passing[passing["Next_Year_PosRank"] != 0]
rushing = rushing[rushing["Next_Year_PosRank"] != 0]
receiving = receiving[receiving["Next_Year_PosRank"] != 0]

passing.reset_index(drop=True, inplace=True)
rushing.reset_index(drop=True, inplace=True)
receiving.reset_index(drop=True, inplace=True)

df_list = [passing, rushing, receiving]
for df in df_list:
    for index in range(df.shape[0]):
        stop = 0
        order = 1
        value = df["Next_Year_PosRank"][index]

        while stop == 0:
            if value <= 10:
                
                df.loc[[index], ['Next_Year_PosRank']] = order * 10
                stop = 1
               
            elif order >= 5:
                df.loc[[index], ['Next_Year_PosRank']] = 50
                stop = 1
                
            else:
                value -= 10
                order += 1
        

In [76]:
passing.to_csv('data/Non_Normalize_Cleaned_Passing.csv', sep=',', index=False, encoding='utf-8')
rushing.to_csv('data/Non_Normalize_Cleaned_Rushing.csv', sep=',', index=False, encoding='utf-8')
receiving.to_csv('data/Non_Normalize_Cleaned_Receiving.csv', sep=',', index=False, encoding='utf-8')

# Normalizing all of the columns using Z-Score normalization

In [77]:
from sklearn.preprocessing import StandardScaler
scaler_standard = StandardScaler()

df_list = [passing, rushing, receiving]
for df in df_list:
    for col in df.columns:
        if (df[col].dtype == float or df[col].dtype == int) and col != "Next_Year_PosRank":
            df[col] = scaler_standard.fit_transform(df[[col]])


In [78]:
passing

,Player,Tm,Age,G,GS,Cmp,Att,Cmp%,Yds,TD,...,team_rush_td,team_rush_yds_per_att,team_rush_fd,team_penalties,team_penalties_yds,team_pen_fd,team_score_pct,team_turnover_pct,team_exp_pts_tot,Next_Year_PosRank
0,Peyton Manning,Colts,-0.319085,1.082989,1.136858,1.402871,1.255367,0.617412,1.333184,1.361130,...,0.547710,-1.131008,0.253217,-0.639106,-1.349864,0.575803,1.924336,-0.756324,0.274543,10
1,Trent Green,Chiefs,1.045728,1.082989,1.136858,1.054672,1.058729,0.258524,1.193848,0.934672,...,3.804551,0.252969,1.061966,-1.447773,-1.101117,-0.104547,0.705557,-1.290563,0.706318,10
2,Marc Bulger,Rams,-0.546554,0.899633,0.982287,1.097309,1.099886,0.267726,1.075290,0.764089,...,1.158368,-1.361670,-0.100610,-0.639106,-1.315316,-0.104547,1.132130,1.317781,-0.959684,10
3,Brad Johnson,Buccaneers,1.500666,1.082989,1.136858,1.225219,1.273659,0.166501,1.054512,1.105255,...,-1.691368,-0.669682,-0.656625,0.916022,1.704202,0.439733,-1.244488,1.066375,-1.435368,20
4,Tom Brady,Patriots,-0.546554,1.082989,1.136858,0.962293,1.077021,-0.008342,0.937787,0.849380,...,-0.877158,-1.822996,-0.403891,0.542791,0.971779,-0.240617,-0.878854,-0.410640,-0.768304,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,Tyrod Taylor,Giants,1.045728,-1.300639,-1.336272,-1.247707,-1.296354,1.353594,-1.239035,-1.027033,...,1.565473,1.406283,2.578369,0.231765,-0.209771,0.575803,0.553210,-1.667674,1.057507,40
937,Brian Hoyer,Patriots,1.955604,-1.667351,-1.181701,-1.254814,-1.305500,2.117382,-1.251869,-1.112325,...,-0.266500,0.252969,-0.050063,0.107355,-0.113036,-0.512757,-0.132353,-0.190659,0.173704,50
938,C.J. Beathard,Jaguars,0.135853,-1.117283,-1.336272,-1.240601,-1.282635,0.304535,-1.253091,-1.112325,...,0.547710,1.175620,0.758685,-0.763517,-1.266948,0.303663,0.842670,-0.127807,1.271608,50
939,Blaine Gabbert,Buccaneers,1.045728,-1.667351,-1.336272,-1.247707,-1.296354,1.353594,-1.256758,-1.027033,...,-1.691368,-1.822996,-1.010452,-0.203671,-0.444700,-0.104547,-0.254231,-0.724898,0.402101,50


In [79]:
print("Final Shapes of DataFrames:\n")
print('Passing Observations:', passing.shape[0], '       Passing Parameters:', passing.shape[1] - 2, '\n')
print('Rushing Observations:', rushing.shape[0], '       Rushing Parameters:', rushing.shape[1] - 2, '\n')
print('Receiving Observations:', receiving.shape[0], '     Receiving Parameters:', receiving.shape[1] - 2, '\n')

Final Shapes of DataFrames:

Passing Observations: 941        Passing Parameters: 66 

Rushing Observations: 1329        Rushing Parameters: 54 

Receiving Observations: 1838      Receiving Parameters: 57 



In [80]:
passing.to_csv('data/Cleaned_Passing.csv', sep=',', index=False, encoding='utf-8')
rushing.to_csv('data/Cleaned_Rushing.csv', sep=',', index=False, encoding='utf-8')
receiving.to_csv('data/Cleaned_Receiving.csv', sep=',', index=False, encoding='utf-8')